In [58]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceBERT 모델 로드

In [59]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510446 -0.77338415  0.59277105 ...  0.579235    0.3268346
  -0.65089685]
 [-0.09361713 -0.18191506 -0.192308   ... -0.03165798  0.30412498
  -0.26793608]]


# 데이터셋 로드

웰니스 대화 스크립트 데이터셋

https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-006

In [66]:
df = pd.read_csv('wellness_dataset.csv')

df.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


# 전처리

In [67]:
df = df.drop(columns=['Unnamed: 0'])


df.head()

,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [96]:
df = df.dropna(subset=['시스템문장1', '시스템문장2', '시스템문장3'])
df.head()



,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3,유저,embedding,similarity
0,중년,여성,"재정,은퇴,노후준비",해당없음,상처,상처,지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.,돈이 없다고 하니 자식이 화를 내서 상처를 받으셨군요.,너무 화가 나. 도와줬던 건 기억 못 하고 더 받을 생각만 하니.,자식들이 받을 생각만 해서 화가 나셨군요. 이 기분을 바꿀 만한 일이 있을까요?,앞으로는 자식보다 나를 생각하며 살아야겠어. 자식 위해 살아도 필요 없다는 말 이제...,이제는 자식보다 나를 생각하며 살려 하시는군요.,지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어...,"[-0.09061889, -0.39553943, -0.03419342, -0.504...",0.370704
1,중년,여성,"재정,은퇴,노후준비",해당없음,상처,상처,친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...,비웃는 친구의 말에 기분이 나쁘셨군요.,아주 나쁜 친구야. 은퇴를 내가 하고 싶어 하는 것도 아닌데 말이야.,은퇴를 비웃는 친구가 나쁘다고 생각하시는군요. 앞으로 어떻게 하실 생각이신가요?,은퇴 후 조금 쉬며 즐기려고 해. 내가 잘할 수 있는 일이 분명 있을 테니까.,은퇴 후에는 쉬면서 즐기려고 할 생각이시군요.,친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...,"[-0.46718258, -0.4527592, 0.20323414, 0.583565...",0.349500
2,중년,여성,"재정,은퇴,노후준비",해당없음,상처,상처,친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했어.,친구분이 은퇴를 한심하게 취급해서 서운하셨군요.,맞아. 그동안 내가 얼마나 힘들게 일했는지 알고 있으면서 그런 말을 하다니.,얼마나 힘들었는지 아는 친구분이 그런 말을 했군요. 기분이 나아질 만한 방법이 있을까요?,감정을 추스르고 나중에 연락을 해야겠어. 지금은 많이 서운해.,감정을 추스르고 나중에 연락을 할 생각이시군요.,친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했...,"[-0.38679314, -0.20814481, -0.36101344, -0.088...",0.633797
3,중년,여성,"재정,은퇴,노후준비",해당없음,상처,상처,그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...,그동안 너무 고생하셨어요. 주변 분들이 많이 걱정해 주시나 봐요.,응. 그렇지만 내 결정에 다들 냉담하게 반응하니 기분이 좋지 않아.,다들 냉담하게 반응해 기분이 좋지 않으시군요. 이 기분을 어떻게 바꿔 볼 수 있을까요?,일단 여행을 다녀 보려고. 그동안 너무 앞만 보고 달려왔어.,여행을 다녀 볼 생각이시군요.,그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...,"[-0.83514875, 0.2009972, -0.0047335196, -0.525...",0.288314
4,중년,여성,"재정,은퇴,노후준비",해당없음,상처,상처,많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋아.,은퇴를 결심했는데 주변에선 섣부르다고 해서 마음이 안 좋으셨군요.,힘들게 결심했는데 다들 반대하니까 조금 더 고민해 보는 게 좋을까 싶어.,조금 더 고민하는 게 좋을지 생각하시는군요. 어떤 선택을 해야 좋을까요?,한 달 정도 더 고민을 해 보려고. 지금 너무 마음이 지쳤어.,마음이 지쳐 한 달 정도 더 고민해 볼 생각이시군요.,많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋...,"[-0.6040759, -0.18082182, 0.30147165, -0.58753...",0.399861


In [97]:
df = df.reset_index(drop=True)
texts = df.loc[0, ['사람문장1', '사람문장2', '사람문장3']].tolist()




In [98]:
encoded = model.encode([
    df.iloc[0]['사람문장1'], 
    df.iloc[0]['사람문장2'], 
    df.iloc[0]['사람문장3']
])



# 유저 대화내용 인코딩

In [99]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df.apply(
    lambda row: model.encode([row['사람문장1'], row['사람문장2'], row['사람문장3']]), 
    axis=1
)


In [103]:
df.to_csv('wellness_dataset.csv', index=False)

# 간단한 챗봇

In [104]:
text = '요즘 너무 힘들어'

embedding = model.encode(text)

In [110]:
df['distance'] = df['embedding'].apply(
    lambda x: cosine_similarity(embedding, np.mean(x, axis=0).reshape(1, -1)).squeeze()
)

In [111]:
# 코드 어딘가에 아래를 추가하여 컬럼명 변경 후 사용하기
df.rename(columns={'distance': 'similarity'}, inplace=True)

answer = df.loc[df['similarity'].idxmax()]

print('구분:', answer.get('구분', 'N/A'))
print('유사한 질문:', answer.get('사람문장2', 'N/A'))
print('챗봇 답변:', answer.get('시스템문장2', 'N/A'))
print('유사도:', answer['similarity'])



구분: N/A
유사한 질문: 20659                 몸도 마음도 요즘 너무 힘들어.
31817    재밌는 일도 없고 몸도 아프고 오래 살 것 같지 않아.
Name: 사람문장2, dtype: object
챗봇 답변: 20659    많이 힘드시겠어요. 지금 이 기분에서 벗어나기 위해 할 수 있는 일은 무엇이 있을까요?
31817                           건강이 좋지 않아 그런 생각을 하시는 건가요?
Name: 시스템문장2, dtype: object
유사도:        similarity  similarity
20659    0.931102    0.754556
31817    0.545555    0.844108


In [112]:
# 'distance' 컬럼명을 'similarity'로 변경
df.rename(columns={'distance': 'similarity'}, inplace=True)

# similarity 컬럼에서 최대값의 인덱스 찾기
answer = df.loc[df['similarity'].idxmax()]

# 결과 출력
print('구분:', answer.get('구분', 'N/A'))
print('사람문장1:', answer.get('사람문장1', 'N/A'))
print('사람문장2:', answer.get('사람문장2', 'N/A'))
print('사람문장3:', answer.get('사람문장3', 'N/A'))
print('시스템문장1:', answer.get('시스템문장1', 'N/A'))
print('시스템문장2:', answer.get('시스템문장2', 'N/A'))
print('시스템문장3:', answer.get('시스템문장3', 'N/A'))
print('유사도:', answer['similarity'])



구분: N/A
사람문장1: 20659    더 가격이 싼 것을 사려고 슈퍼를 몇 군데 돌아다니는 게 힘들어.
31817                        요즘은 사는 게 너무 힘들어.
Name: 사람문장1, dtype: object
사람문장2: 20659                 몸도 마음도 요즘 너무 힘들어.
31817    재밌는 일도 없고 몸도 아프고 오래 살 것 같지 않아.
Name: 사람문장2, dtype: object
사람문장3: 20659    잠깐 음악 들으면서 산책하고 와야겠어.
31817               나 혼자는 힘들어.
Name: 사람문장3, dtype: object
시스템문장1: 20659     여러 군데를 다니셔야 해서 많이 힘드시겠어요.
31817    어떤 연유로 사는 게 힘드신지 여쭤봐도 될까요?
Name: 시스템문장1, dtype: object
시스템문장2: 20659    많이 힘드시겠어요. 지금 이 기분에서 벗어나기 위해 할 수 있는 일은 무엇이 있을까요?
31817                           건강이 좋지 않아 그런 생각을 하시는 건가요?
Name: 시스템문장2, dtype: object
시스템문장3: 20659     좋은 해결책이 나왔으면 좋겠어요.
31817    주변에 도와주실 만한 분이 없나요?
Name: 시스템문장3, dtype: object
유사도:        similarity  similarity
20659    0.931102    0.754556
31817    0.545555    0.844108


In [ ]:
print(df.tail())


       연령  성별 상황키워드    신체질환 감정_대분류 감정_소분류  \
42690  노년  남성    재정  만성질환 무     분노    성가신   
42691  노년  여성    재정  만성질환 무     불안    초조한   
42692  노년  여성    재정  만성질환 무     상처    희생된   
42693  노년  여성  대인관계  만성질환 무     불안  걱정스러운   
42694  노년  여성  대인관계  만성질환 무     상처   배신당한   

                                                   사람문장1  \
42690     나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.   
42691        몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.   
42692   이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.   
42693  몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.   
42694  남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져.   

                                      시스템문장1  \
42690     경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.   
42691  건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.   
42692          노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.   
42693                   가족과의 문제 때문에 속상하시겠어요.   
42694        대인관계에 대한 어려움 때문에 걱정되시고 속상하시겠어요.   

                                                   사람문장2  \
42690              

연령  성별 상황키워드    신체질환 감정_대분류 감정_소분류  \
42690  노년  남성    재정  만성질환 무     분노    성가신   
42691  노년  여성    재정  만성질환 무     불안    초조한   
42692  노년  여성    재정  만성질환 무     상처    희생된   
42693  노년  여성  대인관계  만성질환 무     불안  걱정스러운   
42694  노년  여성  대인관계  만성질환 무     상처   배신당한   

                                                   사람문장1  \
42690     나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.   
42691        몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.   
42692   이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.   
42693  몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.   
42694  남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져.   

                                      시스템문장1  \
42690     경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.   
42691  건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.   
42692          노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.   
42693                   가족과의 문제 때문에 속상하시겠어요.   
42694        대인관계에 대한 어려움 때문에 걱정되시고 속상하시겠어요.   

                                                   사람문장2  \
42690                   아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.   
42691          마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.   
42692  주변 사람들은 다 노후 준비도 잘해두었던데 난 어떻게 해야 할지 모르겠어. 막막하기...   
42693             이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.   
42694             사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야.   

                                 시스템문장2  \
42690      지금 할 수 있는 가장 합리적인 행동은 무엇인가요?   
42691     어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?   
42692  지금의 상황에서 할 수 있는 가장 좋은 행동이 무엇일까요?   
42693  지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?   
42694   어떻게 하면 지금의 상황에 변화를 만들어낼 수 있을까요?   

                                                   사람문장3  \
42690                 노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.   
42691            남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.   
42692      남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.   
42693            함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.   
42694  사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지...   

                                   시스템문장3  \
42690                    좋은 결과 얻으시길 바랄게요.   
42691         남편과 함께하는 좋은 외출 시간 되시길 바랄게요.   
42692     좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.   
42693             지인분들과 좋은 시간 보내셨으면 좋겠어요.   
42694  원하시는 대로 가지고 계시던 걱정이 잘 해결되셨으면 좋겠어요.   

                                                      유저  \
42690  나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....   
42691  몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 마음...   
42692  이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정...   
42693  몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...   
42694  남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사...   

                                               embedding  similarity  \
42690  [[-0.2372779, -0.59531397, 0.22900227, -0.4026...    0.444901   
42691  [[-0.40943107, -0.40100956, 0.37406448, 0.0859...    0.433251   
42692  [[-0.60759276, -0.30602401, 0.072787315, -0.44...    0.336020   
42693  [[-0.7421694, -0.8590092, -0.21469095, -0.3868...    0.607746   
42694  [[-0.53030455, -0.6839293, -0.24093056, -0.045...    0.298523   

       similarity  
42690    0.479816  
42691    0.552983  
42692    0.373097  
42693    0.510914  
42694    0.370273